In [1]:
import json
with open('/Users/Londo/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()



dict_keys(['client-id', 'api-key'])

In [2]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [3]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
type(results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [5]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install tqdm
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 1.5 MB/s eta 0:00:00a 0:00:01


In [6]:
# Specify Search Terms
LOCATION = 'Boston, MA, 02114'
TERM = 'Steak'

In [7]:
# Specify JSON file
JSON_FILE = f"Data/results_in_progress_Steak"
JSON_FILE

'Data/results_in_progress_Steak'

In [8]:
# Check for duplicate file path - from week 2 lessons on LP
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")



[i] Data/results_in_progress_Steak not found. Saving empty list to file.


In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [10]:
# Use yelp_api search query to perform call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
total_results = results['total']
total_results

601

In [12]:
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

31

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [15]:
# Add progress bar
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/31 [00:00<?, ?it/s]

In [16]:
# For loop to call each page - from lesson on LP
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)



  0%|          | 0/31 [00:00<?, ?it/s]

In [17]:
# Convert JSON to dataframe
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fEKzWC1sbfZSElp46FoNag,mooo-restaurant-boston,Mooo Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/N1vvpZ...,False,https://www.yelp.com/biz/mooo-restaurant-bosto...,971,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 42.3583014944831, 'longitude': -7...",[delivery],$$$$,"{'address1': '15 Beacon St', 'address2': '', '...",+16176702515,(617) 670-2515,631.948192
1,P9AJqEYOT--zGRfK7QQlmQ,the-emory-boston,The Emory,https://s3-media1.fl.yelpcdn.com/bphoto/nyCl5R...,False,https://www.yelp.com/biz/the-emory-boston?adju...,119,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 42.35821, 'longitude': -71.06238}","[delivery, pickup]",$$$,"{'address1': '21 Beacon St', 'address2': None,...",+16174306325,(617) 430-6325,619.098491
2,ooaCl61BsK7kJlZnYOq0UQ,abe-and-louies-boston-2,Abe & Louie's,https://s3-media4.fl.yelpcdn.com/bphoto/DdL-R-...,False,https://www.yelp.com/biz/abe-and-louies-boston...,1499,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 42.3491394996001, 'longitude': -7...","[delivery, pickup]",$$$,"{'address1': '793 Boylston St', 'address2': ''...",+16175366300,(617) 536-6300,1862.209421
3,l4YpEuvIm_RqNXleuDIthw,ma-maison-boston,Ma Maison,https://s3-media2.fl.yelpcdn.com/bphoto/aGxVh9...,False,https://www.yelp.com/biz/ma-maison-boston?adju...,353,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 42.3609838, 'longitude': -71.0680...",[delivery],$$,"{'address1': '272 Cambridge St', 'address2': N...",+16177258855,(617) 725-8855,151.339179
4,qVNYCZi2ofv-oe293JsYkg,75-chestnut-boston,75 Chestnut,https://s3-media4.fl.yelpcdn.com/bphoto/lysUij...,False,https://www.yelp.com/biz/75-chestnut-boston?ad...,538,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 42.35695, 'longitude': -71.07097}",[delivery],$$,"{'address1': '75 Chestnut St', 'address2': '',...",+16172272175,(617) 227-2175,663.907465


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
596,TLtykd3vG5_BG-8MP66RHg,kings-dining-and-entertainment-boston-3,Kings Dining & Entertainment,https://s3-media4.fl.yelpcdn.com/bphoto/gy4wfd...,False,https://www.yelp.com/biz/kings-dining-and-ente...,547,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",3.5,"{'latitude': 42.34706, 'longitude': -71.08565}",[delivery],$$,"{'address1': '50 Dalton St', 'address2': '', '...",+16172662695,(617) 266-2695,2231.959271
597,LGBXtRmyE9qbe80L0L2N0Q,dunkin-boston-32,Dunkin',https://s3-media2.fl.yelpcdn.com/bphoto/XDaG94...,False,https://www.yelp.com/biz/dunkin-boston-32?adju...,38,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 42.347318, 'longitude': -71.081891}",[],$,"{'address1': '800 Boylston St Prudential Ctr',...",+16174240158,(617) 424-0158,2043.189728
598,L266shLPDRHyAzWpCtYA3A,dunkin-donuts-boston-73,Dunkin' Donuts,https://s3-media1.fl.yelpcdn.com/bphoto/C1dGXO...,False,https://www.yelp.com/biz/dunkin-donuts-boston-...,5,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",2.0,"{'latitude': 42.3470145, 'longitude': -71.0799...",[],$,"{'address1': '53 Huntington Ave', 'address2': ...",,,1881.823701
599,19xuTm-mcnG4k7rF3ZTmeg,clover-food-truck-boston-16,Clover Food Truck,https://s3-media1.fl.yelpcdn.com/bphoto/HjACIU...,False,https://www.yelp.com/biz/clover-food-truck-bos...,487,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 42.3519287109375, 'longitude': -7...",[],$,"{'address1': '', 'address2': '', 'address3': '...",,,1343.979351
600,py10VCn5raT5mzgI_Oe9Mg,chipotle-mexican-grill-boston-8,Chipotle Mexican Grill,https://s3-media4.fl.yelpcdn.com/bphoto/GSVpQS...,False,https://www.yelp.com/biz/chipotle-mexican-gril...,44,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.0,"{'latitude': 42.351054, 'longitude': -71.045723}","[pickup, delivery]",$,"{'address1': '51 Boston Wharf Rd', 'address2':...",+16174821781,(617) 482-1781,2179.854377


In [18]:
# check for duplicates
final_df.duplicated(subset='id').sum()

0

In [19]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Steak.csv.gz', compression='gzip',index=False)